In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import numba
import numexpr as ne

###### Flow definition #########################################################
#maxIter = 200000 # Total number of time iterations.
maxIter = 200 # Total number of time iterations.
Re      = 220.0  # Reynolds number.
nx = 520; ny = 180; ly=ny-1.0; q = 9 # Lattice dimensions and populations.
cx = nx/4; cy=ny/2; r=ny/9;          # Coordinates of the cylinder.
uLB     = 0.04                       # Velocity in lattice units.
nulb    = uLB*r/Re; omega = 1.0 / (3.0*nulb+0.5); # Relaxation parameter.

###### Lattice Constants #######################################################
c = np.array([(x,y) for x in [0,-1,1] for y in [0,-1,1]]) # Lattice velocities.
t = 1.0/36.0 * np.ones(q)                                   # Lattice weights.

t[np.asarray([np.linalg.norm(ci)<1.1 for ci in c])] = 1.0 / 9.0
t[0] = 4.0 / 9.0

noslip = [c.tolist().index((-c[i]).tolist()) for i in range(q)] 
i1 = np.arange(q)[np.asarray([ci[0]<0  for ci in c])] # Unknown on right wall.
i2 = np.arange(q)[np.asarray([ci[0]==0 for ci in c])] # Vertical middle.
i3 = np.arange(q)[np.asarray([ci[0]>0  for ci in c])] # Unknown on left wall.

###### Function Definitions ####################################################
sumpop = lambda fin: np.sum(fin,axis=0) # Helper function for density computation.

def equilibrium(rho, u):              # Equilibrium distribution function.    
    # Slow Original
    #cu = 3.0 * dot(c, u.transpose(1,0,2))
    
    # No multi-thread
    #cu = 3.0 * einsum('ij,jkl', c, u)
    
    # Multi-thread
    cu = 3.0 * np.tensordot(c, u, axes=([1],[0]))    
    
    u0, u1 = u[0], u[1]
    usqr = ne.evaluate("3.0 / 2.0 * (u0**2 + u1**2)")

    #usqr = 3.0 / 2.0 * (u[0]**2 + u[1]**2)    
        
    feq = (ne.evaluate("rho * (1.0 + cu + 0.5 * (cu ** 2) - usqr)").T * t).T

    #feq = zeros((q,nx,ny))
    #for i in range(q):        
    #    feq[i,:,:] = rho * t[i] * (1.0 + cu[i] + 0.5 * (cu[i] ** 2) - usqr)

    # Slower!    
    #feq = einsum('jk,i,ijk->ijk', rho, t, (1.0 + cu + 0.5 * (cu ** 2) - usqr))
    
    return feq

###### Setup: cylindrical obstacle and velocity inlet with perturbation ########
obstacle = np.fromfunction(lambda x,y: (x-cx)**2+(y-cy)**2<r**2, (nx,ny))
vel = np.fromfunction(lambda d,x,y: (1-d)*uLB*(1.0+1e-4*np.sin(y/ly*2*np.pi)),(2,nx,ny))
feq = equilibrium(np.ones((nx,ny)),vel); fin = feq.copy()
                
def solve(total_iter):
    ###### Main time loop ##########################################################
    for time in xrange(total_iter):
        
        fin[i1,-1,:] = fin[i1,-2,:] # Right wall: outflow condition.
        
        rho = sumpop(fin)           # Calculate macroscopic density and velocity.
        
        # Slow Original
        #u = dot(c.transpose(), fin.transpose((1,0,2)))/rho
        
        # No multi-thread
        #u = einsum('ij,ikl', c, fin) / rho   
        
        # Multi-thread
        u = np.tensordot(c, fin, axes=([0],[0])) / rho        
        
        u[:,0,:] = vel[:,0,:] # Left wall: compute density from known populations.
        rho[0,:] = 1.0 / (1.0 - u[0,0,:]) * (sumpop(fin[i2,0,:]) + 2.0 * sumpop(fin[i1,0,:]))

        feq = equilibrium(rho,u) # Left wall: Zou/He boundary condition.
        fin[i3,0,:] = fin[i1,0,:] + feq[i3,0,:] - fin[i1,0,:]
        
        # Collision step
        #fout = fin - omega * (fin - feq)  
        fout = ne.evaluate("fin - omega * (fin - feq)")

        for i in range(q): 
            fout[i, obstacle] = fin[noslip[i], obstacle]

        for i in range(q): # Streaming step.
            fin[i,:,:] = np.roll(np.roll(fout[i,:,:], c[i,0], axis=0), c[i,1], axis=1)
        
        if (time%100==0): # Visualization
            plt.clf(); plt.imshow(np.sqrt(u[0]**2+u[1]**2).transpose(),cmap=cm.Reds)
            plt.savefig("vel."+str(time/100).zfill(4)+".png")


In [2]:
%timeit solve(200)

1 loops, best of 3: 9.65 s per loop


$$
\frac{\partial \left( \rho c_p T \right)}{\partial t} = \frac{\partial}{\partial x} \left( k \frac{\partial T}{\partial x} \right) + \frac{\partial}{\partial y} \left( k \frac{\partial T}{\partial y} \right) + q
$$

$$
 \frac{\rho c_p T \Delta V -  \rho^o c^o_p T^o \Delta V}{\Delta t} = k_e \frac{T_E - T_P}{\Delta x} A_x - k_w \frac{T_P - T_W}{\Delta x} A_x + k_n \frac{T_N - T_P}{\Delta y} A_y - k_s \frac{T_P - T_S}{\Delta y} A_y + q \Delta V
$$

$$
 T_P = \frac{\Delta t}{\rho c_p \Delta V} \left( \frac{\rho^o c^o_p T^o_P \Delta V}{\Delta t} - k_e \frac{T^o_P - T^o_E}{\Delta x} A_x - k_w \frac{T^o_P - T^o_W}{\Delta x} A_x - k_n \frac{T^o_P - T^o_N}{\Delta y} A_y - k_s \frac{T^o_P - T^o_S}{\Delta y} A_y + q \Delta V \right)
$$

$$
 T_P = \frac{\Delta t}{\rho c_p \Delta V} \left( \frac{\rho^o c^o_p T^o_P \Delta V}{\Delta t} - \frac{k_e A_x}{\Delta x} (T^o_P - T^o_E) - \frac{k_w A_x}{\Delta x} (T^o_P - T^o_W) - \frac{k_n A_y}{\Delta y} (T^o_P - T^o_N) - \frac{k_s A_y}{\Delta y}  (T^o_P - T^o_S) + q \Delta V \right)
$$

$$
\frac{\rho^o c^o_p \Delta V}{\Delta t} - \frac{k_e A_x}{\Delta x} - \frac{k_w A_x}{\Delta x} - \frac{k_n A_y}{\Delta y} - \frac{k_s A_y}{\Delta y} > 0 
$$

$$
\frac{\rho^o c^o_p \Delta V}{\Delta t} > 4 \frac{k A}{\Delta x} 
$$

$$
 \Delta t < \frac{\rho^o c^o_p \Delta V \Delta x}{4 k A} = \frac{\rho^o c^o_p \Delta x^2}{4 k}
$$

In [109]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import numba
import numexpr as ne
import networkx as nx

###### Flow definition #########################################################
Lx, Ly = 10.0, 10.0
ni, nj = 300, 300

dx, dy = Lx / ni, Ly / nj

Ax = dy * 1.0
Ay = dx * 1.0
A = Ax

k = 1.0
rho = 1.0
cp = 1.0
dV = dx * dy * 1.0
q = 100.0

# Condition for positive coefficients 
# dt < rho * c * dx^2 / 2k (for 2D)
dt = 0.9 * rho * cp * (dx ** 2) / (4.0 * k)

def to_continuous(ij):
    i,j = ij
    return i + j * ni
    
zero_I = np.zeros(nj)
zero_J = np.zeros(ni)
last_I = (ni-1) * np.ones(nj)
last_J = (nj-1) * np.ones(ni)
I = np.arange(0,ni)
J = np.arange(0,nj)

left  =  np.array([zero_I,J]).astype(int).T
right =  np.array([last_I,J]).astype(int).T
north =  np.array([I,last_J]).astype(int).T
south =  np.array([I,zero_J]).astype(int).T

graph = nx.grid_2d_graph(ni,nj)

subgraph_left  = graph.subgraph(map(tuple,  left))
subgraph_right = graph.subgraph(map(tuple, right))
subgraph_north = graph.subgraph(map(tuple, north))
subgraph_south = graph.subgraph(map(tuple, south))

T_left  = 10.0
T_right = 10.0
T_north = 10.0
T_south = 10.0

T     = 15.0 * np.ones((ni, nj))
T_old = np.copy(T)

edges = np.array([e for e in graph.edges_iter()])

e0 = edges[:,0]
e1 = edges[:,1]
e0 = [e0[:,0],e0[:,1]]
e1 = [e1[:,0],e1[:,1]]

nodes = np.array([n for n in graph.nodes_iter()])
nodes = [nodes[:,0], nodes[:,1]]

nodes_left  = np.array([n for n in subgraph_left.nodes_iter() ])
nodes_left = [nodes_left[:,0], nodes_left[:,1]]

nodes_right = np.array([n for n in subgraph_right.nodes_iter()])
nodes_right = [nodes_right[:,0], nodes_right[:,1]]

nodes_north = np.array([n for n in subgraph_north.nodes_iter()])
nodes_north = [nodes_north[:,0], nodes_north[:,1]]

nodes_south = np.array([n for n in subgraph_south.nodes_iter()])
nodes_south = [nodes_south[:,0], nodes_south[:,1]]

def solve(total_iter, sources=[]):
    ###### Main time loop ##########################################################
    for time in xrange(total_iter):
        T[:] = 0.0
        
        # apply boundary conditions
        T[nodes_left ] += -2.0 * k * Ax / dx * (T_old[nodes_left ] - T_left )
        T[nodes_right] += -2.0 * k * Ax / dx * (T_old[nodes_right] - T_right)
        T[nodes_north] += -2.0 * k * Ay / dy * (T_old[nodes_north] - T_north)
        T[nodes_south] += -2.0 * k * Ay / dy * (T_old[nodes_south] - T_south)        

        flux = k * A / dx * (T_old[e0] - T_old[e1])  
        np.add.at(T, e0, -flux)
        np.add.at(T, e1,  flux)
        
        T[sources] += q * dV      
        
        T[nodes] += rho * cp * dV * T_old[nodes] / dt
        T[nodes] *= dt / (rho * cp * dV) 
      
        T_old[:] = T[:]  
        
        if (time%100==0): # Visualization
            plt.clf(); plt.imshow(T.transpose(),cmap=cm.jet, vmin=10.0, vmax=15.0)
            plt.savefig("temperature."+str(time/100).zfill(4)+".png")
        
    return T
T = solve(4)
np.set_printoptions(precision=2) 
print T.T

[[ 10.52  10.94  11.38 ...,  11.38  10.94  10.52]
 [ 10.94  12.71  13.43 ...,  13.43  12.71  10.94]
 [ 11.38  13.43  14.45 ...,  14.45  13.43  11.38]
 ..., 
 [ 11.38  13.43  14.45 ...,  14.45  13.43  11.38]
 [ 10.94  12.71  13.43 ...,  13.43  12.71  10.94]
 [ 10.52  10.94  11.38 ...,  11.38  10.94  10.52]]


15.0